## Constants

In [1]:
#P = some prime number greater than alphabet size, MOD = some huge prime number, ALPHABET = alphabet size
P = 257
MOD = int(1e9) + 9
ALPHABET = 256

## KMP

In [2]:
def makePi(pat):
    """
    Params: pattern, text
    Return: PI table, KMP algorithm
    """
    size, i = 0, 1
    n = len(pat)
    
    pi = [0]*n
    
    while i < n:
        if pat[i] == pat[size]:
            size += 1
            pi[i] = size
            i += 1
        else:
            if size != 0:
                size = pi[size-1]
            else:
                pi[i] = 0
                i += 1
    
    return pi

In [3]:
def kmp(pat, text):
    """
    Knuth-Morris-Pratt Algorithm
    Params: pattern, text
    Return: indexes of pattern matches using the PI table
    """
    m, n = len(pat), len(text)
    i, j = 0, 0
    
    pi = makePi(pat)
    
    idxs = []
    while i < n:
        if pat[j] == text[i]:
            i, j = i+1, j+1
        if j == m:
            idxs.append(i-j)
            j = pi[j-1]
        elif i < n and pat[j] != text[i]:
            if j != 0: 
                j = pi[j-1] 
            else: 
                i = i+1
    
    #print(idxs)
    return idxs

## Boyer-Moore

In [4]:
def badChar(pat):
    """
    Params: pattern
    Return: BadCharacter table, Boyer-Moore Algorithm
    """
    n = len(pat)
    shift = [n]*ALPHABET
    
    for i in range(n-1):
        shift[ord(pat[i])] = n - i -1
    
    return shift

In [5]:
def bms(pat, text):
    """
    Boyer-Moore Algorithm
    Params: pattern, text
    Return: indexes of pattern matches, comparisson from right to left
    """
    m, n = len(pat), len(text)
    
    shiftTable = badChar(pat)
    
    i = m-1
    
    idxs = []
    while i < n:
        j = 0
        while j < m and text[i-j] == pat[m-j-1]: 
            j+=1
        if j == m: 
            idxs.append(i-m+1)
        i = i + shiftTable[ord(text[i])]
    
    #print(idxs)
    return idxs

## Rabin-Karp

In [6]:
"""
Reference of optimizations: 
    https://cp-algorithms.com/string/string-hashing.html
    https://cp-algorithms.com/string/rabin-karp.html
"""
def stringHash(pat, text):   
    s, t = len(pat),len(text)
    prime_pow = [1]*max(s, t)
    
    for i in range(1,len(prime_pow)):
        prime_pow[i] = (prime_pow[i-1] * P) % MOD
        
    sHash = 0
    for i in range(s):
        sHash = (sHash + (ord(pat[i]) - ord('A') + 1) * prime_pow[i]) % MOD
    
    return sHash, prime_pow

In [7]:
def rabin_karp(pat, text):
    """
    Rabin-Karp Algorithm
    Params: pattern, text
    Return: indexes of pattern matches, based on rolling hash
    """
    #sHash = hash of pattern, tHash = rolling hash of text, pPow pre-computed prime pow
    sHash, pPow = stringHash(pat, text)
    
    s, t = len(pat),len(text)
    
    tHash = [0] * (t+1)
    for i in range(t):
        tHash[i+1] = (tHash[i] + (ord(text[i]) - ord('A') + 1) * pPow[i]) % MOD
    
    idxs = []
    i = 0
    while i+s-1 < t:
        cur_h = (tHash[i+s] + MOD - tHash[i]) % MOD
        
        if cur_h == (sHash * pPow[i] % MOD):
            idxs.append(i)
        
        i+=1

    #print(idxs)
    return idxs    

## Testes

In [9]:
text = 'ABABDABACDABABCABAB'
pat = 'ABABCABAB'

%timeit kmp(pat, text)
#kmp(pat, text)

%timeit bms(pat, text)
#bms(pat, text)

%timeit rabin_karp(pat, text)
#rabin_karp(pat, text)

6.79 µs ± 101 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
6.19 µs ± 372 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
18.2 µs ± 228 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
